In [1]:
import os
from torchvision.datasets import ImageFolder

data_path = "/opt/ml/data"

train_path = os.path.join(data_path, "train")
val_path = os.path.join(data_path, "val")
test_path = os.path.join(data_path, "test")


# DATA_PATH: "/opt/ml/data/"
# DATASET: "TACO"
# IMG_SIZE: 224
# AUG_TRAIN: "simple_augment_train"
# AUG_TEST: "simple_augment_test"
# AUG_TRAIN_PARAMS: 
# BATCH_SIZE: 64
# EPOCHS: 100
# VAL_RATIO: 0.0
# INIT_LR: 0.1
# FP16: True

# train_dataset, val_dataset, test_dataset = get_dataset(
#         data_path=config["DATA_PATH"],
#         dataset_name=config["DATASET"],
#         img_size=config["IMG_SIZE"],
#         val_ratio=config["VAL_RATIO"],
#         transform_train=config["AUG_TRAIN"],
#         transform_test=config["AUG_TEST"],
#         transform_train_params=config["AUG_TRAIN_PARAMS"],
#         transform_test_params=config.get("AUG_TEST_PARAMS"),
#     )

transform_train = "simple_augment_train"
transform_test = "simple_augment_test"
dataset_name = "TACO"
img_size = 224
transform_train_params = dict()
transform_test_params = dict()

 # preprocessing policies
transform_train = getattr(
    __import__("src.augmentation.policies", fromlist=[""]),
    transform_train,
)(dataset=dataset_name, img_size=img_size, **transform_train_params)


transform_test = getattr(
    __import__("src.augmentation.policies", fromlist=[""]),
    transform_test,
)(dataset=dataset_name, img_size=img_size, **transform_test_params)


train_dataset = ImageFolder(root=train_path, transform=transform_train)
val_dataset = ImageFolder(root=val_path, transform=transform_test)
test_dataset = ImageFolder(root=test_path, transform=transform_test)

len(train_dataset), len(val_dataset), len(test_dataset)

In [ ]:

train_dl, val_dl, test_dl = create_dataloader(data_config)

In [6]:

trainer = TorchTrainer(
    model=model_instance,
    criterion=criterion,
    optimizer=optimizer,
    scheduler=scheduler,
    scaler=scaler,
    device=device,
    model_path=model_path,
    verbose=1,
)

test_loss, test_f1, test_acc = trainer.test(
        model=model_instance, test_dataloader=val_dl if val_dl else test_dl
    )


<All keys matched successfully>

In [ ]:
model_instance

# 모델 저장

In [12]:
import torchvision
import torch

model_instance = torchvision.models.mobilenet_v3_large(pretrained=True, width_mult=1.0,  reduced_tail=False, dilated=False)

model_instance.load_state_dict(
            torch.load("/opt/ml/code/exp/latest2/best.pt")
        )

save_path = os.path.join("/opt/ml/code/model", "mobilenet_v3_best.pt")
torch.save(model_instance, save_path)


In [1]:
import torch
save_path = os.path.join("/opt/ml/code/model", "mobilenet_v3_best.pt")

model = torch.load(save_path)
model

ModuleNotFoundError: No module named 'torchvision.models.mobilenetv3'

In [3]:
import torch


In [ ]:
model_without_ddp.load_state_dict(checkpoint['model'])
optimizer.load_state_dict(checkpoint['optimizer'])
lr_scheduler.load_state_dict(checkpoint['lr_scheduler'])
args.start_epoch = checkpoint['epoch'] + 1

In [17]:
import torchvision

device = torch.device("cuda")
torch.backends.quantized.engine = "qnnpack"
torch.backends.cudnn.benchmark = True

model = torchvision.models.quantization.__dict__["mobilenet_v3_large"](pretrained=True)#, quantize=args.test_only)
model.to(device)

model.fuse_model()
model.qconfig = torch.quantization.get_default_qat_qconfig("qnnpack")
torch.quantization.prepare_qat(model, inplace=True)


checkpoint = torch.load("/opt/ml/code/mobile_q/model_1.pth", map_location='cpu')
model.load_state_dict(checkpoint['model'])


<All keys matched successfully>

In [15]:
model

QuantizableMobileNetV3(
  (features): Sequential(
    (0): ConvBNActivation(
      (0): ConvBn2d(
        3, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False
        (bn): BatchNorm2d(16, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
        (weight_fake_quant): FakeQuantize(
          fake_quant_enabled=tensor([1], device='cuda:0', dtype=torch.uint8), observer_enabled=tensor([1], device='cuda:0', dtype=torch.uint8), quant_min=-128, quant_max=127, dtype=torch.qint8, qscheme=torch.per_tensor_symmetric, ch_axis=-1, scale=tensor([0.0683], device='cuda:0'), zero_point=tensor([0], device='cuda:0')
          (activation_post_process): MovingAverageMinMaxObserver(min_val=-7.443112373352051, max_val=8.70357894897461)
        )
        (activation_post_process): FakeQuantize(
          fake_quant_enabled=tensor([1], device='cuda:0', dtype=torch.uint8), observer_enabled=tensor([1], device='cuda:0', dtype=torch.uint8), quant_min=0, quant_max=255, dtype=torch.qu